### Fixed bug-- rerun and see if matrices changing

In [5]:
import pickle
import os
import sys
import datetime
import pandas as pd
import numpy as np
import nltk.data
from gensim.models import Word2Vec
import multiprocessing
from copy import deepcopy
from sklearn.metrics import roc_auc_score    
from sklearn.cross_validation import StratifiedKFold
import array

/nlp-vm/anaconda2/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [6]:
np.random.seed(10)

In [7]:
import os.path
sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['PYTHONHASHSEED']='10'
#is this working?? no idea

In [8]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

### Read in datafile

In [9]:
# Read data from files
df = pd.read_csv( 'labeledTrainData.tsv', header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0,  delimiter="\t", quoting=3 )

In [10]:
df.columns = ['id', 'boolean_label', 'abstract']

### Word2Vec Algorithm

In [13]:
a = datetime.datetime.now().replace(microsecond=0)

y = df['boolean_label'].values

#set random state
skf = StratifiedKFold(y, n_folds=5, shuffle=True, random_state=10)

i = 1
for train_index, test_index in skf:

    if i == 1:
        
        print("ROUND", i)
        i = i + 1
        #use the indexes to subset the df pandas dataframe
        train1, test1 = df.iloc[train_index], df.iloc[test_index]

        sentences = []  # Initialize an empty list of sentences
        for j in range(len(train1)):
            review = train1['abstract'].iloc[j]
            sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

        print "Parsing sentences from unlabeled set"
        for review in unlabeled_train["review"]:
            sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

        print "Building and training w2v models"
        ## create a w2v learner 
        basemodel = Word2Vec(
            workers=multiprocessing.cpu_count(), # use your cores
            iter=3, # iter = sweeps of SGD through the data; more is better
            hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
            )
        basemodel.build_vocab(sentences) 
        
        # ****** Split the training set into clean sentences
        #
        sentences_pos = []  # Initialize an empty list of sentences
        sentences_neg = []  # Initialize an empty list of sentences

        #here change to include all journal name labels of positive and negative
        inxs_pos = np.where(train1['boolean_label'] == 1)[0].tolist()
        inxs_neg = np.where(train1['boolean_label'] == 0)[0].tolist()

        for inx in inxs_pos:
            review = train1["abstract"].iloc[inx]
            sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

        for inx in inxs_neg:
            review = train1["abstract"].iloc[inx]
            sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

        #train models
        models = [deepcopy(basemodel) for y in range(2)]
        models[0].train(sentences_neg, total_examples=len(sentences_neg) )
        models[1].train(sentences_pos, total_examples=len(sentences_pos) )
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

('ROUND', 1)


/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /nlp-vm/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup 

Parsing sentences from unlabeled set


/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to g

Building and training w2v models
0:09:02


In [49]:
#sources
# https://radimrehurek.com/gensim/models/word2vec.html
# https://groups.google.com/forum/#!topic/gensim/wF3pqnMBptM
# https://rare-technologies.com/word2vec-tutorial/
# http://benjaminbolte.com/blog/2016/keras-gensim-embeddings.html
# http://qua.st/handcoding-neural-network
# http://iamtrask.github.io/2015/07/12/basic-python-network/

## Exploratory finding/getting weight matrices

In [28]:
print type(models[0].vocab)

<type 'dict'>


In [36]:
#get vocab of model
myList = []
for key in models[0].vocab:
    myList.append(key)
print myList[0:10]
print len(myList)
print type(myList)


[u'fawn', u'nunnery', u'sonja', u'askew', u'woods', u'spiders', u'seifer', u'francesco', u'woody', u'comically']
45848
<type 'list'>


In [34]:
myList.sort()
print myList[0:10]

[u'a', u'aa', u'aaa', u'aaab', u'aaargh', u'aag', u'aaja', u'aames', u'aamir', u'aandhi']


In [37]:
myList = []
for key in models[1].vocab:
    myList.append(key)
print myList[0:10]
print myList[45838:45848]
print len(myList)

[u'fawn', u'nunnery', u'sonja', u'askew', u'woods', u'spiders', u'seifer', u'francesco', u'woody', u'comically']
[u'sheri', u'upholds', u'grandiosity', u'cronyn', u'rotting', u'hardboiled', u'emery', u'squeaking', u'expands', u'untalented']
45848


In [25]:
#negative model
print models[0].syn0
print len(models[0].syn0)
print len(models[0].syn0[0])

[[  2.88682848e-01  -6.84787333e-02   7.19660282e-01 ...,   6.88702106e-01
    2.85346091e-01  -1.73179880e-02]
 [ -8.32004368e-01   3.07706803e-01  -4.17696267e-01 ...,  -6.03468418e-01
    4.57414061e-01  -2.87811190e-01]
 [ -4.41641957e-01  -6.29356146e-01   3.11055422e-01 ...,   5.50690174e-01
    1.26547515e-01  -1.55012548e-01]
 ..., 
 [ -3.75039154e-03   3.18863429e-03  -3.25506344e-03 ...,   3.92392278e-03
    3.96958878e-03  -2.97664339e-03]
 [  3.91525356e-03   2.19062567e-04  -4.26600454e-03 ...,   3.90417688e-03
    3.08490521e-03   2.07365444e-03]
 [  1.21666503e-03   8.55820545e-04  -4.38107690e-03 ...,  -7.12921565e-06
   -1.74750190e-03  -4.66067251e-03]]
45848
100


In [56]:
print models[0].syn1
print len(models[0].syn1)
print len(models[0].syn1[0])

[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.00278263 -0.00460976  0.00608409 ..., -0.00094998  0.00658451
  -0.00179398]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.12184668 -0.13286938  0.15408094 ...,  0.15768768 -0.06730247
   0.03437706]
 [ 0.25507295 -0.43136904  0.25583121 ...,  0.01798691 -0.06629085
   0.04001892]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]
45848
100


In [26]:
print models[0].syn0[0]
print models[0].syn0[1]

[  2.88682848e-01  -6.84787333e-02   7.19660282e-01   3.27850021e-02
  -7.02433050e-01   4.02776189e-02   6.40889525e-01  -1.96327060e-01
  -5.43084085e-01  -5.47058702e-01  -1.20406307e-01   1.60259575e-01
   1.21941364e+00  -1.81464091e-01   4.54297885e-02   9.86317620e-02
  -1.33323431e-01  -3.07300180e-01  -3.88504118e-01  -1.53133303e-01
   1.06022852e-02  -4.65012193e-01  -8.25424194e-02   8.80012810e-02
  -2.63207734e-01   2.00721875e-01   1.82375029e-01  -1.88326985e-02
  -6.86327294e-02   1.14784196e-01  -5.68298221e-01  -4.31157559e-01
   7.09036469e-01   7.10079968e-02   5.08382380e-01  -3.84713322e-01
  -8.89201105e-01   2.48976082e-01  -6.07578516e-01   2.74715573e-01
  -8.39516759e-01   4.32502002e-01  -4.44599152e-01  -1.02357358e-01
   2.01037303e-01  -4.11260545e-01   6.75538480e-01  -9.54390168e-02
   2.44938776e-01  -1.84837356e-01  -3.10258538e-01  -1.05104136e+00
   6.06923997e-01  -2.79553253e-02  -4.51465428e-01   1.19402039e+00
   3.73420119e-01  -2.33446866e-01

In [17]:
#positive model
print models[1].syn0


[[  4.98815089e-01   4.54112887e-01   1.90876380e-01 ...,   3.36264133e-01
    3.11656475e-01   4.79288436e-02]
 [ -3.55523050e-01  -7.05757290e-02  -2.27811784e-01 ...,   7.42246270e-01
    1.22729525e-01   3.91686440e-01]
 [  7.11207017e-02   3.32382172e-01  -7.86043890e-03 ...,   9.69603896e-01
   -5.06545961e-01  -1.87086910e-01]
 ..., 
 [ -3.75039154e-03   3.18863429e-03  -3.25506344e-03 ...,   3.92392278e-03
    3.96958878e-03  -2.97664339e-03]
 [  3.91525356e-03   2.19062567e-04  -4.26600454e-03 ...,   3.90417688e-03
    3.08490521e-03   2.07365444e-03]
 [  2.29603741e-02   1.78182516e-02  -3.99268046e-03 ...,   5.15519409e-04
    2.21150145e-02   1.48416404e-02]]


In [18]:
print models[1].syn1

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  2.32950598e-03   7.74592534e-03  -4.06939676e-03 ...,   4.83297696e-03
    2.81180418e-03   4.33527678e-03]
 [ -2.94423808e-04  -8.35394021e-04   6.85739622e-04 ...,   4.39664203e-04
   -1.03778881e-03  -2.46397394e-06]
 ..., 
 [ -3.64709785e-03  -1.78354725e-01  -8.49982053e-02 ...,   2.58107662e-01
   -1.48524955e-01  -1.48581639e-01]
 [ -5.08096665e-02  -2.66497135e-01   1.27625152e-01 ...,  -6.41564839e-03
   -1.36887386e-01   2.66004186e-02]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]


In [35]:
#raw numpy vector of a word
print models[0]['a']
#or access en-masse as a 2D NumPy matrix from model.syn0. 
if models[0]['a'] in models[0].syn0:
    print 'yes'

[-0.44164196 -0.62935615  0.31105542 -0.33523935 -0.24139914 -0.53366935
 -0.53589904 -0.49696487 -0.25657988 -0.04021372 -1.22061765 -0.31305772
  0.12890463  0.2729125   0.12143371  0.14007525 -0.05346049  0.66174066
  0.47774914  0.12089314  0.09315558  0.18877561  0.84103024  1.33529294
 -0.59099913  1.56823397  0.57884836 -0.33871248  1.25823879  1.10080755
  0.61217248 -1.17162812  0.31816494  0.22903992  0.04204141  0.4474912
 -0.13435635 -1.12660885 -0.35122344 -0.09748978 -0.09457557  0.50334752
 -0.58668894  1.11167681  0.48572475 -0.28705376  1.19787383  0.55447316
  1.11031842  0.28093967  0.55512363 -0.30721861  0.61725193  0.53885078
  0.1092468   0.19194022  0.44241491 -0.20178223  0.44433135 -0.16264081
 -0.00451901  0.87856698  0.32355303 -0.78979981 -0.66778165  0.01107274
 -0.23536395 -0.39065754  0.3708007  -0.20732935 -0.73231471 -1.5478704
 -0.21397677  0.07199886 -0.22771484  0.10642073  0.48255414  1.2484864
  0.41594523  0.46829298 -0.57851905  0.59879839 -1.48

In [48]:
np.where(models[0].syn0==models[0]['movie'])

(array([16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]))

In [49]:
models[0].syn0[16]

array([ 0.6609292 ,  0.01555221,  0.73481148, -0.23383325,  1.11858487,
       -0.25723121, -0.7565133 , -0.19132951,  0.62465668,  0.11837119,
       -0.31830201, -0.39404005,  0.62204808, -0.12934224, -0.46964738,
        0.44028723,  0.08260953, -1.48116291, -0.08200017, -0.67697531,
        0.4708699 , -1.4161967 ,  0.58338666, -1.43238592, -1.41162848,
        0.43555543, -0.95845115,  0.20863706,  0.19709915, -0.16422749,
        0.44554555, -0.19723336, -0.42607585,  0.67374074, -0.88089859,
        1.2261343 , -0.02878938, -0.52877027,  1.29530776, -0.64724684,
        0.24425969,  0.05281075,  0.16895212,  0.31853625, -0.22249691,
        0.18122435, -0.72547126, -0.53328723, -0.87055236, -0.65075976,
        0.57691902, -0.05992883, -0.53699625, -0.22737674,  0.10075177,
       -1.08976269,  0.8119396 ,  0.70638978, -0.49699432, -0.2801131 ,
        0.48046473, -0.2001621 , -0.57027853,  0.1861313 ,  0.61592615,
        0.65159291,  0.31207943,  0.2461812 , -0.3610073 ,  0.37

In [ ]:
models[0]['movie']

In [39]:
for i in range(len(models0.syn0)):
    if models[0].syn0[i] == models[0]['fawn']:
        print i

NameError: name 'models0' is not defined

## Going for training on a single review and see how fixed weights change

In [26]:
    models[0]["exception"]

array([ 0.04810371,  0.01240467,  0.3948524 ,  0.28894129,  0.32248172,
       -0.72893578, -0.78935665,  0.45637494,  0.2967777 , -0.23797275,
       -0.29138288,  0.74044579,  0.275803  ,  0.32390475,  0.80199176,
        0.05491541,  0.3200098 , -0.30178878,  0.38178915,  0.32507691,
       -0.00185838,  0.49369264, -0.51240307, -0.07202322,  0.23256029,
       -0.07163434,  0.31362921, -0.19588877,  0.00982205, -0.49936303,
        0.00720923,  0.07985201,  0.53743052,  0.55543411,  0.43858734,
        0.12011249, -0.30702832, -0.39733541, -0.61279953,  0.47660294,
       -0.64364058,  0.1358155 ,  0.28105927,  0.39723414,  0.11340088,
        0.22509968,  0.00439591, -0.13086192,  0.16885729,  0.21151432,
       -0.05161725,  0.2717191 ,  0.12553649,  0.03999346,  0.0807441 ,
       -0.09375352, -0.07193306,  0.77542937, -0.45825517,  0.51046658,
       -0.05519772,  0.11869766,  0.1083831 , -0.26241899,  0.05480757,
        0.02933501, -0.20564111,  0.52109641, -1.0687356 ,  0.33

In [42]:
type(sentences_pos_test[0])
print sentences_pos_test[0]

list

In [20]:
modPos.train(sentences_pos_test[0], total_examples=1 )
modNeg.train(sentences_pos_test[0], total_examples=1 )

213

In [25]:
modPos["exception"]

array([ 0.04810371,  0.01240467,  0.3948524 ,  0.28894129,  0.32248172,
       -0.72893578, -0.78935665,  0.45637494,  0.2967777 , -0.23797275,
       -0.29138288,  0.74044579,  0.275803  ,  0.32390475,  0.80199176,
        0.05491541,  0.3200098 , -0.30178878,  0.38178915,  0.32507691,
       -0.00185838,  0.49369264, -0.51240307, -0.07202322,  0.23256029,
       -0.07163434,  0.31362921, -0.19588877,  0.00982205, -0.49936303,
        0.00720923,  0.07985201,  0.53743052,  0.55543411,  0.43858734,
        0.12011249, -0.30702832, -0.39733541, -0.61279953,  0.47660294,
       -0.64364058,  0.1358155 ,  0.28105927,  0.39723414,  0.11340088,
        0.22509968,  0.00439591, -0.13086192,  0.16885729,  0.21151432,
       -0.05161725,  0.2717191 ,  0.12553649,  0.03999346,  0.0807441 ,
       -0.09375352, -0.07193306,  0.77542937, -0.45825517,  0.51046658,
       -0.05519772,  0.11869766,  0.1083831 , -0.26241899,  0.05480757,
        0.02933501, -0.20564111,  0.52109641, -1.0687356 ,  0.33

In [22]:
modNeg["this"]

array([-0.58965862, -0.66589749,  0.05314846, -0.8292644 , -0.35867512,
       -0.35491529, -0.25217143, -0.37331662, -0.43875304,  0.13109848,
        0.12658736,  0.48886883, -0.07183993, -0.11415164,  0.58525699,
        0.06249871, -0.63482875,  0.77302063,  0.30701616,  0.23499113,
       -0.9644329 , -0.44906121,  0.47221696,  0.00841182,  1.2455678 ,
       -0.53164959, -1.03665268,  0.5315032 ,  1.04942358,  0.51712424,
       -0.54326695, -0.44594124, -0.35676771, -0.16603146,  0.4467237 ,
        1.0807271 ,  0.13193223, -1.65369868, -0.322386  , -0.26922873,
       -1.63668287,  0.64601398, -0.95046562,  0.64283979, -0.20382068,
        0.49586114, -0.19394171, -0.26014313,  0.33725259, -0.36722851,
       -0.2423656 , -0.04495001,  0.07667962, -0.81585813,  0.45386389,
       -0.02599576, -0.8518464 ,  0.60130703, -0.02385483, -0.31239519,
        0.965289  ,  0.79968596, -0.658373  , -0.2202408 , -0.92727858,
       -0.49493632,  0.20068063, -0.22083524, -0.80151153, -0.23

In [24]:
models[1]["this"]

array([-0.58965862, -0.66589749,  0.05314846, -0.8292644 , -0.35867512,
       -0.35491529, -0.25217143, -0.37331662, -0.43875304,  0.13109848,
        0.12658736,  0.48886883, -0.07183993, -0.11415164,  0.58525699,
        0.06249871, -0.63482875,  0.77302063,  0.30701616,  0.23499113,
       -0.9644329 , -0.44906121,  0.47221696,  0.00841182,  1.2455678 ,
       -0.53164959, -1.03665268,  0.5315032 ,  1.04942358,  0.51712424,
       -0.54326695, -0.44594124, -0.35676771, -0.16603146,  0.4467237 ,
        1.0807271 ,  0.13193223, -1.65369868, -0.322386  , -0.26922873,
       -1.63668287,  0.64601398, -0.95046562,  0.64283979, -0.20382068,
        0.49586114, -0.19394171, -0.26014313,  0.33725259, -0.36722851,
       -0.2423656 , -0.04495001,  0.07667962, -0.81585813,  0.45386389,
       -0.02599576, -0.8518464 ,  0.60130703, -0.02385483, -0.31239519,
        0.965289  ,  0.79968596, -0.658373  , -0.2202408 , -0.92727858,
       -0.49493632,  0.20068063, -0.22083524, -0.80151153, -0.23

### Create script

In [14]:
print "Parsing test sentences" #into positive and negative groups so know true label
# read in the test set

sentences_pos_test = []  # Initialize an empty list of sentences
sentences_neg_test = []  # Initialize an empty list of sentences
    
inxs_pos_test = np.where(test1['boolean_label'] == 1)[0].tolist()
inxs_neg_test = np.where(test1['boolean_label'] == 0)[0].tolist()

for inx in inxs_pos_test:
    review = test1["abstract"].iloc[inx]
    sentences_pos_test += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

for inx in inxs_neg_test:
    review = test1["abstract"].iloc[inx]
    sentences_neg_test += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)


Parsing test sentences


In [15]:
#save matrices in new variables to load each time, as well as the negative and positive trained models
neg1 = models[0].syn0
neg2 = models[0].syn1
modNeg = models[0]
modPos = models[1]
pos1 = models[1].syn0
pos2 = models[1].syn1

In [27]:
#need an array of numpy arrays
a = []
a.append(b)
a = np.asarray(a)

In [31]:
print models[0].syn0.shape[0]
print models[0].syn1.shape

45848
(45848, 100)


In [41]:
print models[0].syn0[0].shape
print models[0].syn0[0]
a = []
a.append(models[0].syn0[0])
a.append(models[0].syn0[1])
print len(a)
print a
b=[]
b.append(a)

(100,)
[ 0.91041499  0.23640783 -0.25681743 -0.34276941 -0.37728679  0.13761285
  0.58908045  0.20002657 -0.66374677  0.03647031 -0.61350292  0.07174412
  1.24073875 -0.18461914  0.81289577 -0.20492528 -0.17389943  0.36503348
 -0.69417346  0.18351638 -0.24591836 -0.92419791  0.09821442 -0.22583984
 -0.26916167  1.24837947 -0.2366845  -0.43372598  0.08138654 -0.01597164
 -0.18267538 -0.42040157  0.20611672 -0.18706743 -0.02557867 -0.52907258
 -0.44794378  0.52711022 -0.6502437  -0.10610784 -0.99380797  0.35357845
 -0.13044846 -0.1805222  -0.35491014 -0.30555892 -0.03060734  0.02694359
 -0.51674122 -0.17895103 -0.55113721 -0.63631266  0.53162438 -0.09040273
  0.25773147  0.50415039  0.36851749 -0.20539249  0.3317236  -0.9355638
  0.05584699 -0.24196607 -0.35969019 -0.1017556  -0.22101402 -0.19394358
  0.30151859 -0.50884664  0.68795991  0.5285235   0.11715032 -0.34392527
 -0.43012023 -0.16741316 -0.31368929 -0.25357547  0.91266358 -0.09720628
  0.36325991  1.00752544 -0.42079744 -0.63071

In [20]:
a = datetime.datetime.now().replace(microsecond=0)

all = []
for sent in sentences_pos_test:
    #make copies of models
    copymodPos = modPos
    copymodNeg = modNeg
    a=[]
    #retrain model using a test sentence
    copymodPos.train(sent, total_examples=1 )
    copymodNeg.train(sent, total_examples=1 )
    #for each word....replace the 0's 
    for i in range(models[0].syn0.shape[0]):
        #a.append(np.subtract(neg1[i], copymodNeg.syn0[i]))
        #a.append(np.subtract(neg2[i], copymodNeg.syn1[i]))
        #a.append(np.subtract(pos1[i], copymodPos.syn0[i]))
        #a.append(np.subtract(pos2[i], copymodPos.syn1[i]))
        
        if np.any(np.not_equal(neg1[i],copymodNeg.syn0[i])):
            print "1"
            print i
            print neg1[i]
            print sent
        elif np.any(np.not_equal(neg2[i],copymodNeg.syn1[i])):
            print "2"
            print i
            print neg2[i]
            print sent
        elif np.any(np.not_equal(pos1[i],copymodPos.syn0[i])):
            print "3"
            print i
            print pos1[i]
            print sent
        elif np.any(np.not_equal(pos2[i],copymodPos.syn1[i])):
            print "4"
            print i
            print pos2[i]
            print sent

    #then also append the score to the list*** TODO
    
    #all.append(a)
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'list'

In [43]:
if 1==1:
    print 'yes'

yes


In [ ]:
#repeat for negative sentences*** TODO


### Below is testing of a single sentence

In [13]:
print sentences_pos_test[0]
print sentences_neg_test[0]

[u'although', u'i', u'generally', u'do', u'not', u'like', u'remakes', u'believing', u'that', u'remakes', u'are', u'waste', u'of', u'time', u'this', u'film', u'is', u'an', u'exception']
[u'the', u'film', u'starts', u'with', u'a', u'manager', u'nicholas', u'bell', u'giving', u'welcome', u'investors', u'robert', u'carradine', u'to', u'primal', u'park']


In [14]:
test1[0:5]

,id,boolean_label,abstract
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
11,"""6811_10""",1,"""Although I generally do not like remakes beli..."
16,"""4489_1""",0,"""`The Matrix' was an exciting summer blockbust..."
18,"""3304_10""",1,"""This movie is one among the very few Indian m..."
19,"""9352_10""",1,"""Most people, especially young people, may not..."


In [18]:
#retrain model using a test sentence- TODO: is this training properly? Does it expect a list of lists or something?
models[0].train(sentences_neg_test[0], total_examples=1 )
models[1].train(sentences_neg_test[0], total_examples=1 )

207

Now look at weight matrices again....how much have they changed?

In [20]:
print models[0].syn0[0]
print neg1[0]
#identical? yes

[  4.24046785e-01   1.29570603e-01   2.13253185e-01  -1.90784141e-01
  -5.73329329e-01   3.52431573e-02   5.93789816e-01  -5.15585840e-02
  -5.84989309e-01  -4.20276582e-01  -2.04346031e-01   1.25196889e-01
   5.08659303e-01  -2.69331694e-01   2.03659281e-01   3.48256439e-01
  -1.54727967e-02  -3.32959354e-01  -5.65216839e-01  -1.64941072e-01
  -4.00053203e-01  -8.16992760e-01   1.08800799e-01   5.51972389e-02
  -3.51888418e-01   4.25659657e-01  -2.40370277e-02  -4.25303966e-01
   8.51958618e-02  -1.70676052e-04  -9.36673999e-01   4.07337397e-02
   8.96513045e-01  -3.43396254e-02   6.82746887e-01  -4.06076729e-01
  -5.63397467e-01   9.03431028e-02  -5.78936517e-01   1.51937351e-01
  -1.04451334e+00   4.19589162e-01  -4.74915445e-01  -2.67101616e-01
   5.23371577e-01  -4.71828431e-01   6.78323150e-01  -2.30245084e-01
   9.37828645e-02  -3.24425280e-01  -4.47456926e-01  -6.57814860e-01
   7.76386261e-01   2.15011001e-01  -1.08939886e-01   1.27374268e+00
   2.33052894e-01  -3.00233781e-01

In [19]:
print models[0].syn1[0]
print neg2[0]
#identical? yes

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [21]:
print models[1].syn0[0]
print pos1[0]
#identical? yes

[ 0.49941248  0.40806937 -0.01228495  0.26504648 -0.37615326 -0.4339596
  0.50380939 -0.00587781  0.37700355  0.00215048  0.03396163  0.24712934
  0.49124491  0.1520288   0.15901041 -0.10886002 -0.2772769  -0.13540064
  0.60965848 -0.82724553 -0.57203931  0.95680028  0.84846568  0.04186843
  1.42545223  0.42644352 -0.88878399 -0.10957657  0.30615073 -0.54610467
 -0.5236221   0.42949075 -0.14327484 -0.02344611  0.52613127  0.58127254
 -0.1210241   0.48127803 -0.36878362  0.54414165 -0.12575175  0.17995934
 -0.51886714 -0.84128487 -0.09754249 -0.0780535  -0.73795575 -0.14958926
  0.04497276 -0.26140645 -0.16925676 -0.15003616 -0.21271122 -0.20164749
  0.84195226 -0.0556285  -0.31095383 -1.07853973  0.28056636 -0.48894984
  0.71283567  0.52083665 -0.72072655 -0.75949395 -0.41497079  0.20665865
  0.79719555  0.11628009  0.04627879 -0.12904015 -0.6315614   0.01326748
  0.57716733  0.44534415  0.70483577  0.63104039  0.40520695 -0.44806862
  0.56910145  0.0881054   0.31658611 -0.46437484  0.

In [24]:
print models[1].syn1[0]
print pos2[0]
print type(models[1].syn1[0])
print type(pos2[0])
#identical? yes

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


In [ ]:
#neg1 = models[0].syn0
#neg2 = models[0].syn1
#pos1 = models[1].syn0
#pos2 = models[1].syn1

In [26]:
print np.subtract(models[1].syn0[0], pos1[0])
print np.subtract(models[1].syn0[0], pos1[0]).shape

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
(100,)


In [12]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences